In [2]:
import cv2
import numpy as np

class BallTracker:
    def __init__(self, video_path, output_file, processed_video_output):
        self.video_path = video_path
        self.output_file = output_file
        self.processed_video_output = processed_video_output
        self.quadrant_events = []

    def detect_and_track(self):
        cap = cv2.VideoCapture(self.video_path)
        if not cap.isOpened():
            print("Error: Could not open video.")
            return

        fps = cap.get(cv2.CAP_PROP_FPS)
        width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
        total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

        fourcc = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter(self.processed_video_output, fourcc, fps, (width, height))

        quadrants = [
            (0, 0, width // 2, height // 2),
            (width // 2, 0, width, height // 2),
            (0, height // 2, width // 2, height),
            (width // 2, height // 2, width, height)
        ]

        ball_colors = {
            'yellow': ([20, 100, 100], [30, 255, 255]),
            'white': ([0, 0, 200], [180, 55, 255]),
            'blue': ([100, 150, 0], [140, 255, 255]),
            'red': ([0, 150, 100], [10, 255, 255])
        }
        previous_positions = {color: None for color in ball_colors}

        def get_quadrant(x, y):
            for i, (x1, y1, x2, y2) in enumerate(quadrants):
                if x1 <= x < x2 and y1 <= y < y2:
                    return i + 1
            return None

        def log_event(timestamp, quadrant, color, event_type):
            event_text = f"{timestamp:.2f}, {quadrant}, {color}, {event_type}"
            self.quadrant_events.append(event_text)
            return event_text

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            timestamp = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000.0
            hsv_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            for color, (lower, upper) in ball_colors.items():
                mask = cv2.inRange(hsv_frame, np.array(lower), np.array(upper))
                mask = cv2.erode(mask, None, iterations=2)
                mask = cv2.dilate(mask, None, iterations=2)
                contours, _ = cv2.findContours(mask.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                if contours:
                    c = max(contours, key=cv2.contourArea)
                    ((x, y), radius) = cv2.minEnclosingCircle(c)
                    if radius > 10:
                        cv2.circle(frame, (int(x), int(y)), int(radius), (0, 255, 255), 2)
                        current_quadrant = get_quadrant(int(x), int(y))
                        if previous_positions[color]:
                            prev_x, prev_y = previous_positions[color]
                            previous_quadrant = get_quadrant(prev_x, prev_y)
                            if current_quadrant != previous_quadrant:
                                event_text_exit = log_event(timestamp, previous_quadrant, color, 'Exit')
                                event_text_entry = log_event(timestamp, current_quadrant, color, 'Entry')
                                cv2.putText(frame, f"Exit {previous_quadrant}", (int(x), int(y) - 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                                cv2.putText(frame, f"Entry {current_quadrant}", (int(x), int(y) + 20), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
                        previous_positions[color] = (int(x), int(y))

            out.write(frame)

        cap.release()
        out.release()
        self.save_events()

    def save_events(self):
        with open(self.output_file, 'w') as file:
            file.write("Time, Quadrant Number, Ball Colour, Type\n")
            for event in self.quadrant_events:
                file.write(event + '\n')
        print(f"Events recorded and saved to {self.output_file}")

if __name__ == "__main__":
    video_path = 'AI Assignment video.mp4'
    output_file = 'output_events.txt'
    processed_video_output = 'processed_video.avi'
    tracker = BallTracker(video_path, output_file, processed_video_output)
    tracker.detect_and_track()


Events recorded and saved to output_events.txt
